In [ ]:
import numpy as np

import pyfstat
from pyfstat.helper_functions import get_sft_as_arrays

import utils

# Generating  Noise

Example on how to generate Gaussian noise under different conditions 
(non-stationarity, gaps in the data, multiple detectors)
as well as including monochromatic instrumental artifacts, also known as [spectral lines](https://www.gw-openscience.org/O3/o3speclines/).

0. [The SFT format](#the-sft-format)
0. [Using pyfstat.Writer](#pyfstat.writer)
0. [Non-stationary noise](#non-stationarity)
0. [Gaps](#gaps)
0. [Narrow instrumental artifacts](#lines)

## The SFT format <a name="the-sft-format"></a>

Continuous wave signals are quasi-monochromatic long-standing gravitational waves. Standard models, 
such as those corresponding to the signal produced by non-axisymmetric rapidly-spinning neutron stars, 
produce narrow-banded signals which are Doppler-modulated due to the motion of the detector in the Solar system. 
An accessible and open-access review on the topic can be found [here](https://www.mdpi.com/2218-1997/7/12/474).

Due to the well-localization of a signal in a narrow frequency band, 
Forier transforms are suitable tool to analize this sort of data. 
These signals, however, are not completely monochromatic, 
meaning a full-time Fourier transform could risk smearing a signal across a very broad frequency band.
Instead, we work using *Short Fourier Transforms* (SFTs), which are Fourier transforms of short data segments
(typically around 30 minutes). 

This data format is specific of [LALSuite](https://lscsoft.docs.ligo.org/lalsuite/), 
but PyFstat provides functions (`pyfstat.helper_functions.get_sft_as_arrays`) to read them
as Numpy arrays.

## Using pyfstat.Writer<a name="pyfstat.writer"></a>

The most basic example is to generate Gaussian noise as measured by a single detector.
This operation can be performed using `pyfstat.Writer` as follows:

In [ ]:
# Setup Writer
writer_kwargs ={"label": "single_detector_gaussian_noise",
                "outdir": "Pyfstat_example_data",
                "tstart": 1238166018,     # Starting time of the observating [GPS time]
                "duration": 5 * 86400,    # Duration [seconds]
                "detectors": "H1",        # Detector to simulate, in this case LIGO Hanford
                "F0": 100.,               # Central frequency band [Hz]
                "Band": 1.,               # Frequency band-width [Hz]
                "sqrtSX": 1e-23,          # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,             # Fourier transform time duration
                "SFTWindowType": "tukey", # Window function to compute short Fourier transforms
                "SFTWindowBeta": 0.01,    # Parameter associated to the window function
               }
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()

SFT data is saved at the path specified in `writer.sftfilepath `. This binary format can be opened as numpy arrays using `pyfstat.helper.get_sft_as_arrays` as follows:

In [ ]:
# Read SFT data into numpy arrays and plot real and imaginary parts
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

SFT data could contain different detectors which may be operating during different times.
Thus, `timestamps` and `fourier_data` are dictionaries whose keys correspond to detector names (`H1` for LIGO Hanford, `L1` for LIGO Livingston). `timestamps` labels the time at which the data was taken using GPS seconds, while `fourier_data` contains the Fourier amplitude of such data segment. `frequency`, which is common for all the detectors, is a 1D array labeling the frequency bins.

In [ ]:
utils.plot_real_imag_spectrograms(timestamps["H1"], frequency, fourier_data["H1"]);

Since we are generting zero-meanwhite Gaussian noise, 
there is a simple relation between the single-sided amplitude spectral density `sqrtSX` $\sqrt{S_{\text{n}}}$ 
and the variance of the distribution given by
$$
\sigma^2 = \frac{1}{4} T_{\text{SFT}} S_{\text{n}}
$$
where the two $\frac{1}{2}$ factors are due to the use of a *single-sided* ASD 
and the fact that this standard eviation applies to both the real and imaginary parts of the Fourier transform.

In [ ]:
theoretical_stdev = np.sqrt(0.25 * writer_kwargs["Tsft"]) * writer_kwargs["sqrtSX"]
utils.plot_real_imag_histogram(fourier_data["H1"], theoretical_stdev)

## Non-stationary noise<a name="non-stationarity"></a>

Real data, on the other hand, is hardly ever stationary over long periods of time. 
This is equivalent to having a time-varying amplitud spectral density `sqrtSX`, 
which can be easily implemented by running several instances of `pyfstat.Writer`
and concatenating the resulting file paths using `;`.

In [ ]:
segment_lengths = [5 * 86400, 3 * 86400, 4 * 86400]
segment_sqrtSX = [4e-23, 1e-23, 3e-23]

sft_path = []

# Setup Writer
writer_kwargs = {
                "outdir": "Pyfstat_example_data",
                "tstart": 1238166018,
                "detectors": "H1",     # Detector to simulate, in this case LIGO Hanford
                "F0": 100.,            # Central frequency band [Hz]
                "Band": 1.,            # Frequency band-width [Hz]
                "sqrtSX": 1e-23,       # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,          # Fourier transform time duration
                "SFTWindowType": "tukey",
                "SFTWindowBeta": 0.01,              
}

for segment in range(len(segment_lengths)):
    writer_kwargs["label"] = f"segment_{segment}"
    writer_kwargs["duration"] = segment_lengths[segment]
    writer_kwargs["sqrtSX"] = segment_sqrtSX[segment]

    if segment > 0:
        writer_kwargs["tstart"] += writer_kwargs["Tsft"] + segment_lengths[segment-1] 

    writer = pyfstat.Writer(**writer_kwargs)
    writer.make_data()
    
    sft_path.append(writer.sftfilepath)

sft_path = ";".join(sft_path) # Concatenate different files using ;
frequency, timestamps, fourier_data = get_sft_as_arrays(sft_path)

In [ ]:
utils.plot_real_imag_spectrograms(timestamps["H1"], frequency, fourier_data["H1"]);

# Gaps<a name="gaps"></a>

TO DO: MASK DATA SO ITS PROPERLY PLOTTED

In [ ]:
timestamps = {"H1": 1238166018 + 1800 * np.array([0, 10, 15.5, 30])}

# Setup Writer
writer_kwargs ={"label": "single_detector_gaps",
                "outdir": "Pyfstat_example_data",
                "timestamps": timestamps,
                "F0": 100.,               # Central frequency band [Hz]
                "Band": 1.,               # Frequency band-width [Hz]
                "sqrtSX": 1e-23,          # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,             # Fourier transform time duration
                "SFTWindowType": "tukey", # Window function to compute short Fourier transforms
                "SFTWindowBeta": 0.01,    # Parameter associated to the window function
               }
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
utils.plot_real_imag_spectrograms(timestamps["H1"], frequency, fourier_data["H1"]);

## Narrow instrumental artifacts<a name="lines"></a>

In [ ]:
# Setup Writer
writer_kwargs ={"label": "single_detector_spectral_line",
                "outdir": "Pyfstat_example_data",
                "tstart": 1238166018,     # Starting time of the observating [GPS time]
                "duration": 5 * 86400,    # Duration [seconds]
                "detectors": "H1",        # Detector to simulate, in this case LIGO Hanford
                "F0": 100.,               # Frequency of spectral line [Hz]
                "h0": 1e-24,              # Amplitude of the spectral line
                "phi": 3.1,               # Initial phase of the spectral line
                "Band": 1.,               # Frequency band-width [Hz]
                "sqrtSX": 1e-23,          # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,             # Fourier transform time duration
                "SFTWindowType": "tukey",
                "SFTWindowBeta": 0.01,
               }

writer = pyfstat.LineWriter(**writer_kwargs)
writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
utils.plot_amplitude_phase_spectrograms(timestamps["H1"], frequency, fourier_data["H1"]);

## Multiple detectors

In [ ]:
# Setup Writer
writer_kwargs ={"label": "single_detector_gaussian_noise",
                "outdir": "Pyfstat_example_data",
                "tstart": 1238166018,     # Starting time of the observating [GPS time]
                "duration": 5 * 86400,    # Duration [seconds]
                "detectors": "H1,L1",        # Detector to simulate, in this case LIGO Hanford
                "F0": 100.,               # Central frequency band [Hz]
                "Band": 1.,               # Frequency band-width [Hz]
                "sqrtSX": 1e-23,          # Single-sided Amplitude Spectral Density of the noise
                "Tsft": 1800,             # Fourier transform time duration
                "SFTWindowType": "tukey", # Window function to compute short Fourier transforms
                "SFTWindowBeta": 0.01,    # Parameter associated to the window function
               }
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
for ifo in timestamps.keys():
    fig, ax = utils.plot_real_imag_spectrograms(timestamps[ifo], frequency, fourier_data[ifo]);
    fig.suptitle(ifo)